In [2]:
from u import *
from expres.src.config import Config

from model import Model
import util

%load_ext autoreload
%autoreload 2

In [21]:
util.config = config = Config(Path('results').mk(),
    board_dim=20,
    n_win=5,
                
    res_basic_block=True,
    res_inplanes=[16, 32, 64, 128],
    res_num_blocks=[3, 4, 6, 3],
                
    c_puct=4,
    mcts_iterations=1600,
    mcts_eps=0.25,
    mcts_alpha=0.03, # 0.6 / board_dim
    temp=1,
    move_temp_decay=25,

    lr=1e-2,
    l2_reg=1e-4,

    train_epochs=50000,
    train_batch=1024,
    pred_batch=48,
    early_stop=False,
    time_update_model=30.0,
    epoch_update_model=1,
    epoch_save_model=100,
    epoch_save_games=200,
    max_save=40,

    max_mcts_queue=200000,
    time_reset_eval=60 * 60,
    min_num_states=40960,
    num_mcts_processes=60,
                
    eval_mcts_iterations=5000
).save(force=True)

In [38]:
config = Config(Path('results_small').mk(),
    board_dim=6,
    n_win=4,
                
    res_basic_block=True,
    res_inplanes=[16, 32],
    res_num_blocks=[3, 4],

    c_puct=4,
    mcts_iterations=300,
    mcts_eps=0.1,
    mcts_alpha=0.12, # 0.6 / board_dim
    temp=1,
    move_temp_decay=25,

    lr=1e-2,
    l2_reg=1e-4,

    train_epochs=50000,
    train_batch=1024,
    pred_batch=48,
    early_stop=False,
    time_update_model=30.0,
    epoch_update_model=1,
    epoch_save_model=100,
    epoch_save_games=200,
    max_save=40,

    max_mcts_queue=100000,
    min_num_states=40960,
    num_mcts_processes=60,
                
    eval_mcts_iterations=5000
).save(force=True)

In [30]:
util.config = config = Config(Path('results_tic_tac_toe').mk(),
    board_dim=3,
    n_win=3,
                
    res_basic_block=True,
    res_inplanes=[16],
    res_num_blocks=[3],
                
    c_puct=4,
    mcts_iterations=100,
    mcts_eps=0.1,
    mcts_alpha=0.03, # 0.6 / board_dim
    temp=1,
    move_temp_decay=25,

    lr=1e-2,
    l2_reg=1e-4,

    train_epochs=50000,
    train_batch=1024,
    pred_batch=48,
    early_stop=False,
    time_update_model=30.0,
    epoch_update_model=10,
    epoch_save_model=100,
    epoch_save_games=200,
    max_save=40,

    max_mcts_queue=200000,
    time_reset_eval=60 * 60,
    min_num_states=20480,
    num_mcts_processes=60,
                
    eval_mcts_iterations=5000
).save(force=True)